#QUIZ
> * dataset은 370개의 feature로 이루어진 데이터
* 산탄데르 은행에서 캐글에 경연 의뢰한 데이터로 피처 이름은 임의로 처리됨
* TARGET 정보가 1이면 불만족, 0이면 만족
* https://www.kaggle.com/c/santander-customer-satisfaction/data

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. 데이터 읽기

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/12.앙상블/data/santander-customer-satisfaction/train.csv')
df_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.00,2.07,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,91.56,138.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,40501.08,13501.47,0.0,0.0,0.0,0.0,0.0,0.0,85501.89,85501.89,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/12.앙상블/data/santander-customer-satisfaction/test.csv')
df_test.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.43,6.00,4.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.55,3.00,2.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,57.00,51.45,66.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.87,30.00,21.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


> df_test는 target이 없는 데이터이다. 캐글에 제출용인 것 같다. 따라서 이번 실습에서는 사용하지 않을 것으로 예상된다.

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [ ]:
df_train.isna().sum().sort_values()

ID                           0
imp_venta_var44_ult1         0
imp_venta_var44_hace3        0
imp_trasp_var33_out_ult1     0
imp_trasp_var33_out_hace3    0
                            ..
num_var25                    0
num_var25_0                  0
num_var26                    0
num_op_var41_ult1            0
TARGET                       0
Length: 371, dtype: int64

In [ ]:
df_train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5.477676,0.011458,0.003762,0.0,0.0,0.958024,0.663760,0.000105,0.000026,0.032833,0.028598,0.067522,0.045462,0.052249,0.042936,0.041476,0.010168,0.009997,0.000026,0.000026,0.050855,...,1077.256756,1048.856447,68.275452,9.505287,124.620962,110.026575,3.997023e+03,613.534443,5.703008e+03,4.401002e+03,3639.419939,556.184178,4852.261814,3857.848542,771.227449,162.170439,9.569502e+02,7.509563e+02,0.175324,0.0,0.513023,0.344174,9.117181e+01,3.646318e+01,1.310316e+02,1.092169e+02,0.213071,0.001910,0.253907,0.186630,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,95.268204,319.605516,512.154823,531.897917,950.086398,697.712596,535.473750,953.578624,712.767240,465.391149,0.106425,0.061221,0.0,0.0,0.200535,0.472425,0.010258,0.005129,0.178202,0.166674,0.250925,0.208316,0.222531,0.202714,0.199390,0.100325,0.099486,0.005129,0.005129,0.219703,...,9614.906985,8189.948852,1733.838226,519.389157,2205.249804,1935.305713,3.777314e+04,9292.752726,4.620254e+04,3.550718e+04,26359.174223,7182

In [ ]:
df_train.value_counts('TARGET')

TARGET
0    73012
1     3008
dtype: int64

##1.1 데이터 분리하기

In [ ]:
df_data = df_train.iloc[0:, :-1]
df_labels = df_train['TARGET']
df_labels.head()

0    0
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

#2. 모델 만들기

##2.1 voting

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

lr = LogisticRegression()
knn = KNeighborsClassifier()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

###2.1.1 학습 및 예측

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=7)

start = time.time()
vo.fit(X_train, y_train)
end = time.time()

pred = vo.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

for clf in [lr, knn]:
  clf_name = clf.__class__.__name__

  start = time.time()
  clf.fit(X_train, y_train)
  end = time.time()

  pred = clf.predict(X_test)
  print(accuracy_score(pred, y_test))
  print('time :', end - start)

0.9605367008681925
time : 14.178699493408203
0.9605367008681925
time : 5.4310126304626465
0.9599228273261422
time : 8.730672121047974


> 왠진 모르지만 시간측정이 잘못되엇다. 3가지의 실행이 총 15분가량 걸렸다.

##2.2 배깅

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier()

start = time.time()
rf.fit(X_train, y_train)
end = time.time()

pred = rf.predict(X_test)
print(accuracy_score(y_test, pred))
print(f1_score(y_test, pred))
print('time :', end - start)

0.9584319915811629
0.04627766599597585
time : 16.513957738876343


> 보팅보다 매우 빠르게 실행됬지만, 아주 약간 정확도가 떨어진다.

> f1_score는 그냥 해봣따.

##2.3 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

start = time.time()
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
end = time.time()

pred = gb_clf.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

0.9596597386652636
time : 43.612389087677


> 배깅보다 조금 느리지만 아주 약간 높은 정확도를 보여준다.

##2.4 XG부스팅

In [ ]:
from xgboost import XGBClassifier

start = time.time()
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
end = time.time()

pred = xgb.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

0.9604490046478997
time : 38.74324417114258


> 부스팅보다 빠르고, 시간이 매우 오래걸렸던 보팅과 비슷한 정확도를 보여주고있다.

#3. 스케일링을 통한 모델 성능 높이기
> 보팅의 로지스틱리그레션의 경우 스케일링하면 성능이 향상되니, StandardScaler를 이용해 표준화 해보자

> 또 다른 모델들도 원본값과 어떤 차이가 있는지 살펴보자

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
standard_scaler = scaler.fit_transform(df_data)

X_train, X_test, y_train, y_test = train_test_split(standard_scaler, df_labels, test_size=0.3, random_state=7)

##3.1 voting (scaled)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

lr = LogisticRegression(max_iter=1000)
knn = KNeighborsClassifier()

start = time.time()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')
vo.fit(X_train, y_train)
end = time.time()

pred = vo.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

for clf in [lr, knn]:
  clf_name = clf.__class__.__name__

  start = time.time()
  clf.fit(X_train, y_train)
  end = time.time()

  pred = clf.predict(X_test)
  print(accuracy_score(pred, y_test))
  print('time :', end - start)

0.9603174603174603
time : 23.67161202430725
0.9596597386652636
time : 15.127609491348267
0.9577304218188196
time : 8.715234279632568


> 원본 데이터 기록
- 0.9605367008681925
- time : 14.178699493408203
- 0.9605367008681925
- time : 5.4310126304626465
- 0.9599228273261422
- time : 8.730672121047974


> 정확도는 비슷하지만 아주 약간 떨어졌고, 속도가 오히려 느려졌다. 보팅은 원본 데이터로 진행하는것이 좋을 것 같다.

##3.2 배깅 (scaled)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier()

start = time.time()
rf.fit(X_train, y_train)
end = time.time()

pred = rf.predict(X_test)
print(accuracy_score(y_test, pred))
print('time :', end - start)

0.95834429536087
0.044265593561368215
time : 14.13201904296875


> 원본데이터기록
- 0.9584319915811629

##3.3 부스팅 (scaled)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

start = time.time()
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
end = time.time()

pred = gb_clf.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

0.9596158905551171
time : 78.82276964187622


> 원본데이터기록
- 0.9596597386652636
- time : 43.612389087677

##3.4 XG부스팅 (scaled)

In [ ]:
from xgboost import XGBClassifier

start = time.time()
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
end = time.time()

pred = xgb.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

0.9604490046478997
time : 41.45889115333557


> 원본데이터 기록
- 0.9604490046478997
- time : 38.74324417114258

> 전체적으로 정확도는 비슷하거나 매우조금 하락했고, 속도는 약간 느려졌다. 

> 원본 데이터로 진행하는게 좋을 것 같다.

#4. 하이퍼 파라미터

> 표준화된 데이터보다 원본데이터의 결과가 더 좋으니 원본데이터로 진행한다.

> 실행시간이 너무 오래걸려 최대한 간결하게 테스트한다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=7)

##4.1 voting 하이퍼 파라미터

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
import time

lr = LogisticRegression(max_iter=1000)
knn = KNeighborsClassifier()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')

def get_best_hyper_parameter(Cost_list, n_neighbors_list):
  # 모델id + '__' + 파라미터
  params = {
      'lr_id__C' : Cost_list,
      'knn_id__n_neighbors' : n_neighbors_list,
      'knn_id__weights' : ['uniform', 'distance'],
      'knn_id__metric' : ['euclidean', 'mangattan', 'minkowski']
  }

  start = time.time()
  grid_cv = GridSearchCV(vo, param_grid=params, scoring='accuracy', cv=2)
  grid_cv.fit(X_train, y_train)
  end = time.time()
  print('\n===============================================')
  print('최고 점수 : ', grid_cv.best_score_)
  print('최적의 파라미터 : ', grid_cv.best_params_)
  print('time : ', end - start)
  print('===============================================\n')

In [ ]:
Cost_list=[0.001, 0.01, 0.1, 1, 10, 25, 50]
n_neighbors_list= list(range(1, 100, 10))
get_best_hyper_parameter(Cost_list, n_neighbors_list)

##4.2 배깅 하이퍼 파라미터

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time
rf = RandomForestClassifier()

def get_best_hyper_parameter(n_estimators_list, max_depth_list, min_samples_split_list, min_samples_leaf_list):
  params = {
      'n_estimators' : n_estimators_list,
      'max_depth' : max_depth_list,
      'min_samples_split' : min_samples_split_list,
      'min_samples_leaf' : min_samples_leaf_list
  }

  start = time.time()
  grid_cv = GridSearchCV(rf, param_grid=params, scoring='accuracy', cv=2)
  grid_cv.fit(X_train, y_train)
  end = time.time()
  print('\n===============================================')
  print('최고 점수 : ', grid_cv.best_score_)
  print('최적의 파라미터 : ', grid_cv.best_params_)
  print('time : ', end - start)
  print('===============================================\n')

In [ ]:
n_estimators_list= [100, 300, 500]
max_depth_list= list(range(5, 15, 5))
min_samples_split_list= list(range(2, 15, 5))
min_samples_leaf_list= list(range(1, 15, 5))

get_best_hyper_parameter(n_estimators_list, max_depth_list, min_samples_split_list, min_samples_leaf_list)


최고 점수 :  0.9604427406321645
최적의 파라미터 :  {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
time :  877.1794009208679



> 최고 점수 :  0.9604427406321645

> 최적의 파라미터 :  {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

> time :  877.1794009208679


In [ ]:
n_estimators_list= [100]
max_depth_list= list(range(6, 15, 2))
min_samples_split_list= list(range(2, 7, 2))
min_samples_leaf_list= list(range(1, 6, 2))

get_best_hyper_parameter(n_estimators_list, max_depth_list, min_samples_split_list, min_samples_leaf_list)

> 최고 점수 :  0.9604427406321645

> 최적의 파라미터 :  {'max_depth': 12, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

> time :  329.65931940078735

In [ ]:
n_estimators_list= [100]
max_depth_list= [11, 12, 13]
min_samples_split_list= [2, 3]
min_samples_leaf_list= [1, 2]

get_best_hyper_parameter(n_estimators_list, max_depth_list, min_samples_split_list, min_samples_leaf_list)


최고 점수 :  0.9604615326793701
최적의 파라미터 :  {'max_depth': 13, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 100}
time :  109.04492402076721



> 최고 점수 :  0.9604615326793701

> 최적의 파라미터 :  {'max_depth': 13, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 100}

> time :  109.04492402076721

##4.3 부스팅 하이퍼 파라미터

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

gb_clf = GradientBoostingClassifier()

def get_best_hyper_parameter(n_estimators_list, learning_rate):
  params = {
      'n_estimators' : n_estimators_list,
      'learning_rate' : learning_rate,
  }

  start = time.time()
  grid_cv = GridSearchCV(gb_clf, param_grid=params, scoring='accuracy', cv=2)
  grid_cv.fit(X_train, y_train)
  end = time.time()
  print('\n===============================================')
  print('최고 점수 : ', grid_cv.best_score_)
  print('최적의 파라미터 : ', grid_cv.best_params_)
  print('time : ', end - start)
  print('===============================================\n')

In [ ]:
n_estimators_list= [100, 200, 300]
learning_rate= [0.001, 0.01, 0.1, 1, 10, 100]

get_best_hyper_parameter(n_estimators_list, learning_rate)


최고 점수 :  0.9603863644905476
최적의 파라미터 :  {'learning_rate': 0.001, 'n_estimators': 100}
time :  1591.5935797691345



> 최고 점수 :  0.9603863644905476

> 최적의 파라미터 :  {'learning_rate': 0.001, 'n_estimators': 100}

> time :  1591.5935797691345

##4.4 XG부스팅 하이퍼 파라미터

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier()
evals = [(X_test, y_test)]

def get_best_hyper_parameter(n_estimators_list, learning_rate, max_depth_list, subsample_list):
  params = {
      'n_estimators' : n_estimators_list,
      'learning_rate' : learning_rate,
      'max_depth' : max_depth_list,
      'subsample' : subsample_list
  }

  start = time.time()
  grid_cv = GridSearchCV(xgb, param_grid=params, scoring='accuracy', cv=2)
  grid_cv.fit(X_train, y_train, early_stopping_rounds=50, eval_set=evals, eval_metric='logloss')
  
  end = time.time()
  print('\n===============================================')
  print('최고 점수 : ', grid_cv.best_score_)
  print('최적의 파라미터 : ', grid_cv.best_params_)
  print('time : ', end - start)
  print('===============================================\n')

In [ ]:
#n_estimators_list= [100]
#learning_rate= [0.1, 0.4, 0.7, 1.0]
#max_depth_list= list(range(3, 10, 3))
#subsample_list= [0.1, 0.4, 0.7, 1.0]

n_estimators_list= [100]
learning_rate= [0.1]
max_depth_list= [3]
subsample_list= [0.1]
get_best_hyper_parameter(n_estimators_list, learning_rate, max_depth_list, subsample_list)

[0]	validation_0-logloss:0.612771
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.546899
[2]	validation_0-logloss:0.49156
[3]	validation_0-logloss:0.44458
[4]	validation_0-logloss:0.404892
[5]	validation_0-logloss:0.370825
[6]	validation_0-logloss:0.341585
[7]	validation_0-logloss:0.316726
[8]	validation_0-logloss:0.294506
[9]	validation_0-logloss:0.275134
[10]	validation_0-logloss:0.25874
[11]	validation_0-logloss:0.244297
[12]	validation_0-logloss:0.231463
[13]	validation_0-logloss:0.220294
[14]	validation_0-logloss:0.210434
[15]	validation_0-logloss:0.201974
[16]	validation_0-logloss:0.194684
[17]	validation_0-logloss:0.188247
[18]	validation_0-logloss:0.182647
[19]	validation_0-logloss:0.177466
[20]	validation_0-logloss:0.172953
[21]	validation_0-logloss:0.169029
[22]	validation_0-logloss:0.165369
[23]	validation_0-logloss:0.16201
[24]	validation_0-logloss:0.159405
[25]	validation_0-logloss:0.157136
[26]	validation_0-logloss:0.154988
[

#5. 최적의 파라미터

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=7)

##3.1 voting (scaled)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

lr = LogisticRegression(max_iter=1000)
knn = KNeighborsClassifier()

start = time.time()
vo = VotingClassifier([('lr_id', lr), ('knn_id', knn)], voting='soft')
vo.fit(X_train, y_train)
end = time.time()

pred = vo.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

for clf in [lr, knn]:
  clf_name = clf.__class__.__name__

  start = time.time()
  clf.fit(X_train, y_train)
  end = time.time()

  pred = clf.predict(X_test)
  print(accuracy_score(pred, y_test))
  print('time :', end - start)

0.9603174603174603
time : 23.67161202430725
0.9596597386652636
time : 15.127609491348267
0.9577304218188196
time : 8.715234279632568


> 원본 데이터 기록
- 0.9605367008681925
- time : 14.178699493408203
- 0.9605367008681925
- time : 5.4310126304626465
- 0.9599228273261422
- time : 8.730672121047974


##3.2 배깅 (scaled)

> 최고 점수 :  0.9604615326793701

> 최적의 파라미터 :  {'max_depth': 13, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 100}

> time :  109.04492402076721

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(max_depth=13, min_samples_leaf=1, min_samples_split=3, n_estimators=100)

start = time.time()
rf.fit(X_train, y_train)
end = time.time()

pred = rf.predict(X_test)
print(accuracy_score(y_test, pred))
print('time :', end - start)

0.95834429536087
0.044265593561368215
time : 14.13201904296875


> 원본데이터기록
- 0.9584319915811629

##3.3 부스팅 (scaled)

> 최고 점수 :  0.9603863644905476

> 최적의 파라미터 :  {'learning_rate': 0.001, 'n_estimators': 100}

> time :  1591.5935797691345

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

start = time.time()
gb_clf = GradientBoostingClassifier(learning_rate=0.001, n_estimators=100)
gb_clf.fit(X_train, y_train)
end = time.time()

pred = gb_clf.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

0.9596158905551171
time : 78.82276964187622


> 원본데이터기록
- 0.9596597386652636
- time : 43.612389087677

##3.4 XG부스팅 (scaled)

In [ ]:
from xgboost import XGBClassifier

start = time.time()
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
end = time.time()

pred = xgb.predict(X_test)
print(accuracy_score(pred, y_test))
print('time :', end - start)

0.9604490046478997
time : 41.45889115333557


> 원본데이터 기록
- 0.9604490046478997
- time : 38.74324417114258

#강사님 코드

##1. 파일 읽기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/12.앙상블/data/santander-customer-satisfaction/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/12.앙상블/data/santander-customer-satisfaction/test.csv')
df_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.00,2.07,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,91.56,138.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,40501.08,13501.47,0.0,0.0,0.0,0.0,0.0,0.0,85501.89,85501.89,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [5]:
df_train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5.477676,0.011458,0.003762,0.0,0.0,0.958024,0.663760,0.000105,0.000026,0.032833,0.028598,0.067522,0.045462,0.052249,0.042936,0.041476,0.010168,0.009997,0.000026,0.000026,0.050855,...,1077.256756,1048.856447,68.275452,9.505287,124.620962,110.026575,3.997023e+03,613.534443,5.703008e+03,4.401002e+03,3639.419939,556.184178,4852.261814,3857.848542,771.227449,162.170439,9.569502e+02,7.509563e+02,0.175324,0.0,0.513023,0.344174,9.117181e+01,3.646318e+01,1.310316e+02,1.092169e+02,0.213071,0.001910,0.253907,0.186630,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,95.268204,319.605516,512.154823,531.897917,950.086398,697.712596,535.473750,953.578624,712.767240,465.391149,0.106425,0.061221,0.0,0.0,0.200535,0.472425,0.010258,0.005129,0.178202,0.166674,0.250925,0.208316,0.222531,0.202714,0.199390,0.100325,0.099486,0.005129,0.005129,0.219703,...,9614.906985,8189.948852,1733.838226,519.389157,2205.249804,1935.305713,3.777314e+04,9292.752726,4.620254e+04,3.550718e+04,26359.174223,7182

In [7]:
df_test.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.00000,75818.000000,75818.000000,75818.000000,75818.000000,75818.0,75818.0,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,...,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,7.581800e+04,7.581800e+04,7.581800e+04,7.581800e+04,75818.000000,75818.000000,75818.000000,75818.000000,7.581800e+04,75818.000000,7.581800e+04,75818.000000,75818.000000,75818.0,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.0,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,7.581800e+04
mean,75874.830581,-1579.955011,33.138832,83.164329,74.312894,123.136448,4.578517,7.666855,0.470645,0.672444,3.635893,69.734376,115.469593,67.439982,110.967136,137.214620,67.910627,111.63958,140.850513,4.917151,0.012108,0.003917,0.0,0.0,0.957002,0.663747,0.000092,0.000026,0.033686,0.029887,0.068612,0.045359,0.053206,0.043486,0.042154,0.010763,0.010578,0.000066,0.000066,0.051874,...,902.370806,1056.867678,1060.014504,60.764337,9.361781,118.186622,104.805798,4.359742e+03,7.048762e+02,6.059647e+03,4.745336e+03,3581.184666,501.124806,4909.473875,3869.986824,8.167134e+02,176.592698,9.869169e+02,771.302026,0.199158,0.0,1.218142,0.708650,36.323577,6.204746,57.528068,47.688547,0.138711,0.0,0.195140,0.160595,12.438559,1.327408,17.469991,12.674349,63.597839,11.404505,95.973025,70.504319,1.173863e+05
std,43882.370827,39752.473358,12.932000,1694.873886,364.211245,606.431562,133.383326,239.701416,34.028605,42.336668,129.089127,330.563913,540.054605,517.066406,726.368848,697.823263,521.377495,732.61944,717.878305,322.535540,0.109369,0.062466,0.0,0.0,0.202853,0.472430,0.009608,0.005136,0.180421,0.170278,0.252794,0.208091,0.224446,0.203949,0.200941,0.103184,0.102304,0.008121,0.008121,0.221775,...,7477.578055,9787.585269,8138.430011,1697.257696,510.104537,2240.333363,1815.886791,4.549291e+04,1.184441e+04,5.116131e+04,4.000231e+04,25634

> var3에 -999999값이 존재한다. 이것은 test에도 존재하니 어떠한 (설문 데이터라면 무응답)의미가 있는 데이터라고 생각할 수 있다.

> 수치 데이터 확인 결과 -999999라는 이상치를 확인했다. 뿐만 아니라 각 feature별로 4 분위수의 범위를 한참 벗어나는 수치들이 나타나기 때문에 대부분 이상치로 보여 지기는 한다. 예를들어  var15의 경우 4분위수는 모두 0으로 나타나지만 최대값만21000으로 나타난다. 현재 데이터는 고객 만족에 대한 데이터이고, 만약 var15의 값이 설문에 대한 내용이라고 판단된다면, 최대값이 5점 최소값이 0점이라는 가정하에 모두 최고점을 주었다고 생각할 수도 있다.(물론 가정이다.)

> 어쨋든 현재 데이터들이 정확치 않은 상황에서 이상치 제이터를 변환하는 것보다는 있는 그대로의 값으로 사용하는 것이 좋을 듯 하다.

##2. 데이터 전처리

###2.1 데이터 확인

In [8]:
df_train['var3'].value_counts().sort_index()

-999999      116
 0            75
 1           105
 2         74165
 3           108
           ...  
 228           1
 229           1
 231           1
 235           1
 238           1
Name: var3, Length: 208, dtype: int64

In [9]:
df_test['var3'].value_counts().sort_index()

-999999      120
 0            76
 1            99
 2         73962
 3           107
           ...  
 227           2
 229           1
 230           2
 233           1
 238           1
Name: var3, Length: 199, dtype: int64

> 현재 데이터가 -999999를 제외한 모든 값들이 0이상의 양수로 표현되며 순차적인 데이터로 보인다. -999999의 의미가 정확히 무엇인지는 알 수 없다. 예를 들어 설문으로 생각한다면 무응답 테이터로 판단할 수도 있고, 에러나 예외 관련된 데이터일 수도 있다. -999999의 데이터를 전체 흐름에 벗어나지 않도록 수정한다면 -1정도로 하여 간격을 맞추는것도 한 방법일 것이다.

###2.2 이상치 처리

In [11]:
df_train['var3'].replace(-999999, -1, inplace=True)
df_test['var3'].replace(-999999, -1, inplace=True)
df_train['var3'].value_counts().sort_index()

-1        116
 0         75
 1        105
 2      74165
 3        108
        ...  
 228        1
 229        1
 231        1
 235        1
 238        1
Name: var3, Length: 208, dtype: int64

> -999999의 데이터 개수가 116개였고, 2를 제외한 다른 값들을 보더라도 100개 안밖으로 나타남으로 다른 데이터에 영향을 미치지 않도록 -999999를 -1로 변경한다.

###2.3 불필요한 feature 처리

In [12]:
df_train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,2.711905,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5.477676,0.011458,0.003762,0.0,0.0,0.958024,0.663760,0.000105,0.000026,0.032833,0.028598,0.067522,0.045462,0.052249,0.042936,0.041476,0.010168,0.009997,0.000026,0.000026,0.050855,...,1077.256756,1048.856447,68.275452,9.505287,124.620962,110.026575,3.997023e+03,613.534443,5.703008e+03,4.401002e+03,3639.419939,556.184178,4852.261814,3857.848542,771.227449,162.170439,9.569502e+02,7.509563e+02,0.175324,0.0,0.513023,0.344174,9.117181e+01,3.646318e+01,1.310316e+02,1.092169e+02,0.213071,0.001910,0.253907,0.186630,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,9.449044,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,95.268204,319.605516,512.154823,531.897917,950.086398,697.712596,535.473750,953.578624,712.767240,465.391149,0.106425,0.061221,0.0,0.0,0.200535,0.472425,0.010258,0.005129,0.178202,0.166674,0.250925,0.208316,0.222531,0.202714,0.199390,0.100325,0.099486,0.005129,0.005129,0.219703,...,9614.906985,8189.948852,1733.838226,519.389157,2205.249804,1935.305713,3.777314e+04,9292.752726,4.620254e+04,3.550718e+04,26359.174223,7182.642532,

- ID 는 다른 값에 영향을 줄 수 있기 때문에 제거하는게 좋다.

In [13]:
df_train.drop('ID', axis=1, inplace=True)
df_test.drop('ID', axis=1, inplace=True)
df_train.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.00,2.07,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,91.56,138.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,...,40501.08,13501.47,0.0,0.0,0.0,0.0,0.0,0.0,85501.89,85501.89,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


##3. 모델 만들기

###3.1 데이터 분리

In [17]:
df_labels = df_train['TARGET']
df_data = df_train.drop(columns='TARGET', axis=1)
labels = df_labels.values.reshape(-1)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_data, labels, test_size=0.3, random_state=62)

###3.2 XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# n_estimators 는 500개의 분류기 사용
xgb_clf = XGBClassifier(n_estimators=500)

#early_stopping_rounds은 데이터 검증 결과가 100회 동안 범위를 유지하면 중지한.
#eval_metric은 성능 평가 지표에 auc를 이용함
#auc는 이진 분류기일 경우 사용되며 현재 고객의 만족 불만족 두 가지를 평가하는 것이기 떄문에 사용함
#eval_set은 성능평가 데이터셋
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_test, y_test)])

In [25]:
from sklearn.metrics import roc_auc_score

# predict_proba는 확률 정보를 갖게되며 [:, 1]은 두 가지 값 중 1의 값을 사용한다는 의미이다.
# 보통 1이 될 확률을 구하게 된다.
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score

0.8346427754938953

###3.3 파이퍼 파라미터

In [28]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_clf = XGBClassifier(n_estimators=100)

def get_best_params():
  params = {
      'max_depth' : [3, 6],

      #min_sample_leaf과 유사한값, 과적합 조절에 사용
      'min_child_weight' : [1, 3],

      #max_feature와 유사, feature가 많을 경우 과적합 조절을 위해 사용, 확률 데이터
      'colsample_bytree' : [0.5, 0.75],
  }
  grid_cv = GridSearchCV(xgb_clf, param_grid = params, cv=3)
  grid_cv.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_test, y_test)])
  
  print('\n=========================================================================')
  print('최적의 파라미터 : ', grid_cv.best_params_)
  print('최고점수 : ', grid_cv.best_score_)

  xgb_roc_score = roc_auc_score(y_test, grid_cv.predict_proba(X_test)[:, 1])
  print('AUX : ', xgb_roc_score)

In [29]:
get_best_params()

[0]	validation_0-auc:0.712542
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.791501
[2]	validation_0-auc:0.781087
[3]	validation_0-auc:0.794721
[4]	validation_0-auc:0.801498
[5]	validation_0-auc:0.793614
[6]	validation_0-auc:0.793878
[7]	validation_0-auc:0.802763
[8]	validation_0-auc:0.804621
[9]	validation_0-auc:0.804409
[10]	validation_0-auc:0.807921
[11]	validation_0-auc:0.811626
[12]	validation_0-auc:0.813339
[13]	validation_0-auc:0.810313
[14]	validation_0-auc:0.807925
[15]	validation_0-auc:0.811001
[16]	validation_0-auc:0.813755
[17]	validation_0-auc:0.814566
[18]	validation_0-auc:0.814562
[19]	validation_0-auc:0.815134
[20]	validation_0-auc:0.814807
[21]	validation_0-auc:0.814471
[22]	validation_0-auc:0.812062
[23]	validation_0-auc:0.816573
[24]	validation_0-auc:0.818919
[25]	validation_0-auc:0.819255
[26]	validation_0-auc:0.817808
[27]	validation_0-auc:0.820057
[28]	validation_0-auc:0.821281
[29]	validation_0-auc:0.821126
[30]	validation

KeyboardInterrupt: ignored

> 최적의 파라미터 :  {'colsample_bytree': 0.75, 'max_depth': 3, 'min_child_weight': 1}

> 최고점수 :  0.960123275829669

In [30]:
import numpy as np

#n_estimators는 1000으로 증가시키고, learning_rate=0.02로 감소
# reg_alpha : 피처 개수가 많을 떄 적용을 검토, 클수록 과적합 감소
xgb_clf = XGBClassifier(n_estimators=1000, learning_rate=0.02, max_depth=3, min_child_weight=1, colsample_bytree=0.5, reg_alpha=0.03)

#조기 중단 파라미터 값은 200으로 설정
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, eval_metric="auc", eval_set=[(X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
print(f'ROC-AUC : {np.round(xgb_roc_score, 4)}')

[0]	validation_0-auc:0.700203
Will train until validation_0-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.788782
[2]	validation_0-auc:0.779697
[3]	validation_0-auc:0.782541
[4]	validation_0-auc:0.790253
[5]	validation_0-auc:0.784562
[6]	validation_0-auc:0.785781
[7]	validation_0-auc:0.786332
[8]	validation_0-auc:0.792604
[9]	validation_0-auc:0.792308
[10]	validation_0-auc:0.792966
[11]	validation_0-auc:0.793717
[12]	validation_0-auc:0.794356
[13]	validation_0-auc:0.794952
[14]	validation_0-auc:0.794747
[15]	validation_0-auc:0.800763
[16]	validation_0-auc:0.801838
[17]	validation_0-auc:0.802299
[18]	validation_0-auc:0.80283
[19]	validation_0-auc:0.801915
[20]	validation_0-auc:0.803049
[21]	validation_0-auc:0.802637
[22]	validation_0-auc:0.801643
[23]	validation_0-auc:0.802477
[24]	validation_0-auc:0.802909
[25]	validation_0-auc:0.802969
[26]	validation_0-auc:0.802442
[27]	validation_0-auc:0.803047
[28]	validation_0-auc:0.803343
[29]	validation_0-auc:0.803343
[30]	validation_